# Поиск аномалий

Методы обнаружения аномалий, как следует из названия, позволяют находить необычные объекты в выборке. Но что такое "необычные" и совпадает ли это определение у разных методов?

Начнём с поиска аномалий в текстах: научимся отличать вопросы о программировании от текстов из 20newsgroups про религию.

Подготовьте данные: в обучающую выборку возьмите 20 тысяч текстов из датасета Stack Overflow, а тестовую выборку сформируйте из 10 тысяч текстов со Stack Overflow и 100 текстов из класса soc.religion.christian датасета 20newsgroups (очень пригодится функция `fetch_20newsgroups(categories=['soc.religion.christian'])`). Тексты про программирование будем считать обычными, а тексты про религию — аномальными.

In [11]:
import os
import numpy as np
import pandas as pd
import random
from datasets import load_dataset
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups

so = pd.read_parquet("stackoverflow-posts-00000-of-00058.parquet")

so = pd.read_parquet(
    "stackoverflow-posts-00000-of-00058.parquet",
    engine="pyarrow"
)

texts = (so["Title"].fillna("") + " " + so["Body"].fillna(""))[:30000].values


train_so = texts[:20000]
test_so = texts[20000:30000]

religion_texts = []

religion_texts = []
folder = "20news-bydate-train/soc.religion.christian"

for filename in os.listdir(folder):
    with open(os.path.join(folder, filename), "r", encoding="latin1") as f:
        religion_texts.append(f.read())

print("Всего религиозных текстов:", len(religion_texts))

test_religion = religion_texts[:100]

X_train_texts = train_so
X_test_texts = list(test_so) + test_religion
y_test = np.array([0]*len(test_so) + [1]*len(test_religion))

print("Train:", len(X_train_texts))
print("Test:", len(X_test_texts))


Всего религиозных текстов: 599
Train: 20000
Test: 10100


**(1 балл)**

Проверьте качество выделения аномалий (precision и recall на тестовой выборке, если считать аномалии положительным классов, а обычные тексты — отрицательным) для IsolationForest. В качестве признаков используйте TF-IDF, где словарь и IDF строятся по обучающей выборке. Не забудьте подобрать гиперпараметры.

In [19]:
vectorizer = TfidfVectorizer(
    max_features=10000,
    stop_words='english'
)
X_train = vectorizer.fit_transform(X_train_texts)
X_test = vectorizer.transform(X_test_texts)

clf = IsolationForest(
    n_estimators=200,
    max_samples='auto',
    contamination=len(test_religion)/X_test.shape[0],
    random_state=42
)
clf.fit(X_train)

y_pred = clf.predict(X_test)

y_pred_binary = np.where(y_pred == -1, 1, 0)

precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")


Precision: 0.044
Recall: 0.050


**(5 баллов)**

Скорее всего, качество оказалось не на высоте. Разберитесь, в чём дело:
* посмотрите на тексты, которые выделяются как аномальные, а также на слова, соответствующие их ненулевым признакам
* изучите признаки аномальных текстов
* посмотрите на тексты из обучающей выборки, ближайшие к аномальным; действительно ли они похожи по признакам?

Сделайте выводы и придумайте, как избавиться от этих проблем. Предложите варианты двух типов: (1) в рамках этих же признаков (но которые, возможно, будут считаться по другим наборам данных) и методов и (2) без ограничений на изменения. Реализуйте эти варианты и проверьте их качество.

In [20]:
feature_names = np.array(vectorizer.get_feature_names_out())

anomaly_indices = np.where(y_pred_binary == 1)[0]

print("Первые 5 аномальных текстов с ключевыми словами:")
for idx in anomaly_indices[:5]:
    text = X_test_texts[idx]
    tfidf_row = X_test[idx].toarray().flatten()  # длина = feature_names.shape[0]
    nonzero_indices = np.where(tfidf_row > 0)[0]  # индексы только в диапазоне 0..max_features-1
    top_words = feature_names[nonzero_indices][:10]  # безопасно
    print(f"\nТекст {idx}: {text[:200]}...")
    print("Слова признаков:", top_words)

Первые 5 аномальных текстов с ключевыми словами:

Текст 28:  
1. The question is not really related to checked vs. unchecked debate, the same applies to both exception types.
2. Between the point where the ConstraintViolationException is thrown and the point, ...
Слова признаков: ['abort' 'action' 'additional' 'applies' 'available' 'calls' 'cancel'
 'care' 'catch' 'checked']

Текст 271:  The book 'Real-Time Collision Detection' by Christer Ericson (ISBN: 1-55860-732-3) is a recent (2005) and widely praised book which should give you some good answers.

It starts with a basic primer o...
Слова признаков: ['2005' 'actually' 'algorithms' 'aligned' 'allows' 'answers' 'avoid'
 'aware' 'away' 'axis']

Текст 313:  A tool is a big help.

However, there are times when you can't use a tool: the heap dump is so huge it crashes the tool, you are trying to troubleshoot a machine in some production environment to whi...
Слова признаков: ['access' 'act' 'allocated' 'allocation' 'aren' 'arr' 'backward

### Эксперимент только с изменением датасета

In [21]:
#code here
import numpy as np

# Вычисляем TF-IDF для train
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train_texts)

# Находим "редкие" строки: среднее TF-IDF > threshold
row_means = X_train_tfidf.mean(axis=1).A.flatten()
threshold = np.percentile(row_means, 5)  # нижние 5% редких
filtered_indices = np.where(row_means > threshold)[0]

X_train_filtered = X_train_tfidf[filtered_indices]
clf = IsolationForest(
    n_estimators=200,
    max_samples='auto',
    contamination=len(test_religion)/X_test.shape[0],
    random_state=42
)
clf.fit(X_train_filtered)
X_test_tfidf = vectorizer.transform(X_test_texts)
y_pred = clf.predict(X_test_tfidf)
y_pred_binary = np.where(y_pred == -1, 1, 0)

from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}")


Precision: 0.098, Recall: 0.090


### Эксперимент с любыми изменениями

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score
import numpy as np

vectorizer = TfidfVectorizer(
    max_features=25000,
    ngram_range=(1,3),
    stop_words=None
)

X_train = vectorizer.fit_transform(X_train_texts)
X_test = vectorizer.transform(X_test_texts)

clf = IsolationForest(
    n_estimators=500,
    max_samples='auto',
    contamination=len(test_religion)/X_test.shape[0],
    random_state=42
)
clf.fit(X_train)

y_pred = clf.predict(X_test)
y_pred_binary = np.where(y_pred == -1, 1, 0)

# Метрики
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")


Precision: 0.235
Recall: 0.230


Подготовьте выборку: удалите столбцы `['id', 'date', 'price', 'zipcode']`, сформируйте обучающую и тестовую выборки по 10 тысяч домов.

Добавьте в тестовую выборку 10 новых объектов, в каждом из которых испорчен ровно один признак — например, это может быть дом из другого полушария, из далёкого прошлого или будущего, с площадью в целый штат или с таким числом этажей, что самолётам неплохо бы его облетать стороной.

Посмотрим на методы обнаружения аномалий на более простых данных — уж на табличном датасете с 19 признаками всё должно работать как надо!

Скачайте данные о стоимости домов: https://www.kaggle.com/harlfoxem/housesalesprediction/data

In [ ]:
#code here

**Задание 9. (2 балла)**

Примените IsolationForest для поиска аномалий в этих данных, запишите их качество (как и раньше, это precision и recall). Проведите исследование:

Нарисуйте распределения всех признаков и обозначьте на этих распределениях объекты, которые признаны аномальными.

In [ ]:
#code here